In [1]:
import threading
import time
import subprocess
import os
import re
import psutil
import torch
import tempfile
import tf2onnx
import tensorflow as tf
import coremltools as ct
import coremltools as ct
import onnx
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

2024-07-31 20:08:07.699060: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
scikit-learn version 1.3.2 is not supported. Minimum required version: 0.17. Maximum required version: 1.1.2. Disabling scikit-learn conversion API.
XGBoost version 2.1.0 has not been tested with coremltools. You may run into unexpected errors. XGBoost 1.4.2 is the most recent version that has been tested.
Torch version 2.2.2 has not been tested with coremltools. You may run into unexpected errors. Torch 2.2.0 is the most recent version that has been tested.
/opt/anaconda3/envs/env_name/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/sta

In [5]:

# 加载 MNIST 数据集
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# 调整输入数据的形状和范围
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# 将标签转换为 one-hot 编码
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# 创建 CNN 模型
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# 训练模型
model.fit(X_train, y_train, batch_size=128, epochs=2, verbose=1, validation_data=(X_test, y_test))
# 评估模型
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

#model.save('./mnist_models/mnist_cnn_model.h5')
model.save('./models_train/mnist_cnn_model.h5')

Epoch 1/2
469/469 [==============================] - 9s 18ms/step - loss: 0.2979 - accuracy: 0.9091 - val_loss: 0.0561 - val_accuracy: 0.9813
Epoch 2/2
469/469 [==============================] - 8s 17ms/step - loss: 0.0937 - accuracy: 0.9723 - val_loss: 0.0396 - val_accuracy: 0.9875
Test loss: 0.03961620852351189
Test accuracy: 0.987500011920929


In [7]:
thread_output = {}
duration = 0
# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None
        # time.sleep(0.1)

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    print("Saving model...")
    # --------------------------------------------------------------------------------
    start_time = time.time()
    # 保存 Keras 模型为 HDF5 格式    
    for i in range(0, 10):
        model.save("mnist_cnn_model.h5") 
    
    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save TensorFlow SavedModel: {duration} seconds')
    # --------------------------------------------------------------------------------
    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
                # time.sleep(0.1)
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    # process.wait(timeout=5)  # 尝试在5秒内等待进程结束
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

stop_event = threading.Event()
model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread3.start()
thread1.start()
thread2.start()
# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')] # or
                  # line.startswith('Combined Power (CPU + GPU + ANE):')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)


# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-h5.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 
# print("Extracted numbers:", numbers)

delta_time = duration * 2 / filtered_lines_count
# print(delta_time)

numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")


Saving model...
Subprocess started.
Time taken to save TensorFlow SavedModel: 0.3518099784851074 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_cnn/output-mnist-cnn-h5.txt
0
[]


ZeroDivisionError: float division by zero

CNN-MNIST-H5-inference duration

In [2]:
thread_output = {}
duration = 0
# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None
        # time.sleep(0.1)

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    
    # 测量保存模型的时间
    start_time = time.time()
    # 保存 Keras 模型为 HDF5 格式
    for i in range(0, 10):
        model.save("mnist_cnn_model.h5")
    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save TensorFlow SavedModel: {duration} seconds')

    # 测量推理时间
    X_test_sample = X_test[:10]  # 选择前10个样本进行推理
    start_time_inference = time.time()
    n_time = 32 # 计数器
    for i in range(n_time): # 重复操作n遍
        model.predict(X_test_sample)
    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples {n_time} times: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
                # time.sleep(0.1)
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 载入数据和模型
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

stop_event = threading.Event()
model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread3.start()
thread1.start()
thread2.start()
# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')] # or
                  # line.startswith('Combined Power (CPU + GPU + ANE):')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-h5.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 

delta_time = duration * 2 / filtered_lines_count
# print(delta_time)

numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")

Saving model...
Subprocess started.
Time taken to save TensorFlow SavedModel: 0.3404529094696045 seconds
1/1 [==============================] - 0s 51ms/step
Time taken for inference on 10 samples 32 times: 2.2049 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_cnn/output-mnist-cnn-h5.txt
260
['CPU Power: 2515 mW', 'GPU Power: 0 mW', 'CPU Power: 4350 mW', 'GPU Power: 0 mW', 'CPU Power: 7284 mW', 'GPU Power: 0 mW', 'CPU Power: 7077 mW', 'GPU Power: 0 mW', 'CPU Power: 4511 mW', 'GPU Power: 0 mW', 'CPU Power: 6466 mW', 'GPU Power: 0 mW', 'CPU Power: 7826 mW', 'GPU Power: 0 mW', 'CPU Power: 4921 mW', 'GPU Power: 65 mW', 'CPU Power: 5183 mW', 'GPU Power: 0 mW', 'CPU Power: 1723 mW', 'GPU Power: 64 mW', 'CPU Power: 4299 mW', 'GPU Power: 63 mW', 'CPU Power: 6703 mW', 'GPU Power: 0 mW', 'CPU Power: 4869 mW', 'GPU Power: 0 mW', 'CPU Power: 5597 mW', 'GPU Power: 0 mW', 'CPU Power: 8058 mW', 'GPU Power: 62 mW', 'CPU Power: 6542 mW', 'GPU Power: 249

cnn-pth

In [10]:
thread_output = {}
duration = 0
# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None
        # time.sleep(0.1)

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    print("Saving model...")
    # --------------------------------------------------------------------------------
    start_time = time.time()

    # 保存整个模型
    for i in range(5):
        torch.save(model, 'pytorch_model.pth')

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save PyTorch model: {duration} seconds')
    # --------------------------------------------------------------------------------
    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
                # time.sleep(0.1)
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    # process.wait(timeout=5)  # 尝试在5秒内等待进程结束
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

stop_event = threading.Event()
model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread3.start()
thread1.start()
thread2.start()
# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')] # or
                  # line.startswith('Combined Power (CPU + GPU + ANE):')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)


# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-pth.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 
# print("Extracted numbers:", numbers)

delta_time = duration * 2 / filtered_lines_count
# print(delta_time)

numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")


Saving model...
Subprocess started.
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dropout
.........vars
......flatten
.........vars
......max_pooling2d
.........vars
......max_pooling2d_1
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2024-07-25 20:17:55        

+inference duration pth

In [3]:
import time
import threading
import psutil
import subprocess
import tempfile
import os
import re
import tensorflow as tf
import torch

thread_output = {}
duration = 0
inference_duration = 0  # 添加用于存储推理时间的变量

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    start_time = time.time()

    # 保存整个模型
    for i in range(10):
        torch.save(model, 'pytorch_model.pth')

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save PyTorch model: {duration} seconds')

    # 测量推理时间
    start_time = time.time()

    # 示例推理代码：假设输入数据为随机生成的
    input_data = tf.random.normal([1, 28, 28, 1])  # 根据你的模型调整输入形状
    n_time = 32
    for i in range(n_time):
        model.predict(input_data)

    end_time = time.time()
    inference_duration = end_time - start_time
    print(f'Time taken for inference on {len(input_data)} samples, {n_time} times: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 定义推理测试的线程函数
def test_inference(stop_event, model):
    global inference_duration
    print("Testing inference...")
    start_time = time.time()

    # 示例推理代码：假设输入数据为随机生成的
    input_data = tf.random.normal([1, 28, 28, 1])  # 根据你的模型调整输入形状
    model.predict(input_data)

    end_time = time.time()
    inference_duration = end_time - start_time
    print("hahaha")
    print(f'Time taken for inference: {inference_duration} seconds')
    stop_event.set()  # 触发停止其他线程

stop_event = threading.Event()
model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread4 = threading.Thread(target=test_inference, args=(stop_event, model))

thread3.start()
thread1.start()
thread2.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 最后再做推理
# thread4.start()
# thread4.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
lines = content.split('\n')
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')]

filtered_content = '\n'.join(filtered_lines)

# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-pth.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.2f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption} mV")
else:
    print("No filtered lines to process.")


Saving model...
Subprocess started.
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dropout
.........vars
......flatten
.........vars
......max_pooling2d
.........vars
......max_pooling2d_1
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2024-07-31 20:10:26        

In [7]:
thread_output = {}
duration = 0
inference_duration = 0  # 添加用于存储推理时间的变量

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    start_time = time.time()

    # 保存整个模型为 PyTorch 格式
    for i in range(5):
        torch.save(model, 'pytorch_model.pth')

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save PyTorch model: {duration:.2f} seconds')

    # 测量推理时间
    X_test_sample = X_test[:10]  # 选择前10个样本进行推理
    start_time_inference = time.time()
    model.predict(X_test_sample)
    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 载入数据和模型
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

stop_event = threading.Event()
model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread4 = threading.Thread(target=test_inference, args=(stop_event, model))

thread4.start()
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()
thread4.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
lines = content.split('\n')
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')]

filtered_content = '\n'.join(filtered_lines)

# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-pth.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")

Testing inference...Saving model...

Subprocess started.
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dropout
.........vars
......flatten
.........vars
......max_pooling2d
.........vars
......max_pooling2d_1
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
1/1 [==============================] - 0s 141ms/step
Time taken for inference: 0.3086659908294678 seconds
Keras model archive saving:
File Name                  

CNN-Mnist-ONNX

In [44]:
thread_output = {}
duration = 0
# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None
        # time.sleep(0.1)

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    print("Saving model...")
    # --------------------------------------------------------------------------------
    start_time = time.time()
     # 使用 tf2onnx 保存模型为ONNX格式
    spec = (tf.TensorSpec((None, 28, 28), tf.float32, name="input"),)
    output_path = "mnist_cnn_model.onnx"
    
    for i in range(20):
        model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=output_path)

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save ONNX model: {duration} seconds')
    # --------------------------------------------------------------------------------
    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
                # time.sleep(0.1)
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    # process.wait(timeout=5)  # 尝试在5秒内等待进程结束
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

stop_event = threading.Event()
model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread3.start()
thread1.start()
thread2.start()
# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')] # or
                  # line.startswith('Combined Power (CPU + GPU + ANE):')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)


# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-onnx.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 
# print("Extracted numbers:", numbers)

delta_time = duration * 2 / filtered_lines_count
# print(delta_time)

numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")

Saving model...
Subprocess started.
Time taken to save ONNX model: 18.647239208221436 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_cnn/output-mnist-cnn-onnx.txt
1954
['CPU Power: 6416 mW', 'GPU Power: 0 mW', 'CPU Power: 5679 mW', 'GPU Power: 0 mW', 'CPU Power: 5735 mW', 'GPU Power: 0 mW', 'CPU Power: 5037 mW', 'GPU Power: 0 mW', 'CPU Power: 5370 mW', 'GPU Power: 0 mW', 'CPU Power: 6413 mW', 'GPU Power: 46 mW', 'CPU Power: 6894 mW', 'GPU Power: 88 mW', 'CPU Power: 6573 mW', 'GPU Power: 93 mW', 'CPU Power: 5811 mW', 'GPU Power: 443 mW', 'CPU Power: 5362 mW', 'GPU Power: 246 mW', 'CPU Power: 4483 mW', 'GPU Power: 52 mW', 'CPU Power: 5497 mW', 'GPU Power: 0 mW', 'CPU Power: 5235 mW', 'GPU Power: 0 mW', 'CPU Power: 6253 mW', 'GPU Power: 54 mW', 'CPU Power: 5667 mW', 'GPU Power: 278 mW', 'CPU Power: 5798 mW', 'GPU Power: 64 mW', 'CPU Power: 5217 mW', 'GPU Power: 0 mW', 'CPU Power: 6151 mW', 'GPU Power: 0 mW', 'CPU Power: 5484 mW', 'GPU Pow

Subprocess finished.


+inference duration onnx

In [5]:
thread_output = {}
duration = 0
inference_duration = 0

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None
        # time.sleep(0.1)

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    # --------------------------------------------------------------------------------
    start_time = time.time()
    # 使用 tf2onnx 保存模型为ONNX格式
    spec = (tf.TensorSpec((None, 28, 28), tf.float32, name="input"),)
    output_path = "mnist_cnn_model.onnx"
    
    for i in range(10):
        model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=output_path)

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save ONNX model: {duration} seconds')
    # --------------------------------------------------------------------------------
    
    # 测量推理时间
    X_test_sample = X_test[:10]  # 选择前10个样本进行推理
    start_time_inference = time.time()
    for i in range(1):
        model.predict(X_test_sample)
        end_time_inference = time.time()
        inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
                # time.sleep(0.1)
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    # process.wait(timeout=5)  # 尝试在5秒内等待进程结束
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 载入数据和模型
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

stop_event = threading.Event()
model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread3.start()
thread1.start()
thread2.start()
# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')] # or
                  # line.startswith('Combined Power (CPU + GPU + ANE):')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-onnx.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 
# print("Extracted numbers:", numbers)

delta_time = duration * 2 / filtered_lines_count
# print(delta_time)

numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")

Saving model...
Subprocess started.
Time taken to save ONNX model: 2.404927968978882 seconds
1/1 [==============================] - 0s 60ms/step
Time taken for inference on 10 samples: 0.0879 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_cnn/output-mnist-cnn-onnx.txt
252
['CPU Power: 4648 mW', 'GPU Power: 0 mW', 'CPU Power: 5315 mW', 'GPU Power: 0 mW', 'CPU Power: 3587 mW', 'GPU Power: 133 mW', 'CPU Power: 3245 mW', 'GPU Power: 250 mW', 'CPU Power: 4036 mW', 'GPU Power: 0 mW', 'CPU Power: 4827 mW', 'GPU Power: 0 mW', 'CPU Power: 4173 mW', 'GPU Power: 0 mW', 'CPU Power: 4601 mW', 'GPU Power: 0 mW', 'CPU Power: 5182 mW', 'GPU Power: 0 mW', 'CPU Power: 5422 mW', 'GPU Power: 0 mW', 'CPU Power: 4988 mW', 'GPU Power: 0 mW', 'CPU Power: 3792 mW', 'GPU Power: 0 mW', 'CPU Power: 2377 mW', 'GPU Power: 0 mW', 'CPU Power: 3479 mW', 'GPU Power: 0 mW', 'CPU Power: 4004 mW', 'GPU Power: 0 mW', 'CPU Power: 5071 mW', 'GPU Power: 0 mW', 'CPU Power: 509

CNN-Mnist-core ML

In [19]:
thread_output = {}
duration = 0

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    print("Saving model...")
    # --------------------------------------------------------------------------------
    start_time = time.time()
    # 将TensorFlow Keras模型转换为Core ML模型
    for i in range(20):
        try:
            input_shape = ct.Shape(shape=(1, 28, 28, 1))
            # 指定输入层的名称和形状
            input_feature = ct.TensorType(name="conv2d_input", shape=input_shape)
            # 转换模型时指定输入形状
            mlmodel = ct.convert(model, inputs=[input_feature])
            #mlmodel.save(f"mnist_cnn_model_{i}.mlpackage")
        except Exception as e:
            print(f"Error saving model {i}: {e}")
    
    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save Core ML: {duration} seconds')
    # --------------------------------------------------------------------------------
    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

stop_event = threading.Event()
model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))

thread3.start()
thread1.start()
thread2.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-coreml.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 
# print("Extracted numbers:", numbers)

delta_time = duration * 2 / filtered_lines_count
# print(delta_time)

numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")


When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html


Saving model...
Subprocess started.


Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 1792.05 passes/s]
When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 1846.83 passes/s]
When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your

Time taken to save Core ML: 32.40747690200806 seconds
Resource monitoring finished.
Subprocess finished.
Content saved to ./mnist_models/output_cnn/output-mnist-cnn-coreml.txt
4158
['CPU Power: 4596 mW', 'GPU Power: 0 mW', 'CPU Power: 4049 mW', 'GPU Power: 0 mW', 'CPU Power: 6225 mW', 'GPU Power: 0 mW', 'CPU Power: 3642 mW', 'GPU Power: 511 mW', 'CPU Power: 7246 mW', 'GPU Power: 126 mW', 'CPU Power: 4571 mW', 'GPU Power: 125 mW', 'CPU Power: 5046 mW', 'GPU Power: 66 mW', 'CPU Power: 3923 mW', 'GPU Power: 0 mW', 'CPU Power: 7579 mW', 'GPU Power: 446 mW', 'CPU Power: 4551 mW', 'GPU Power: 442 mW', 'CPU Power: 3472 mW', 'GPU Power: 0 mW', 'CPU Power: 3662 mW', 'GPU Power: 63 mW', 'CPU Power: 3505 mW', 'GPU Power: 0 mW', 'CPU Power: 5298 mW', 'GPU Power: 249 mW', 'CPU Power: 5457 mW', 'GPU Power: 0 mW', 'CPU Power: 4793 mW', 'GPU Power: 378 mW', 'CPU Power: 6871 mW', 'GPU Power: 0 mW', 'CPU Power: 5434 mW', 'GPU Power: 340 mW', 'CPU Power: 3272 mW', 'GPU Power: 200 mW', 'CPU Power: 3310 mW

inference duration core ml

In [7]:
thread_output = {}
duration = 0
inference_duration = 0  # 添加推理时间变量

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    start_time = time.time()

    # 将TensorFlow Keras模型转换为Core ML模型
    for i in range(2):
        try:
            input_shape = ct.Shape(shape=(1, 28, 28, 1))
            input_feature = ct.TensorType(name="conv2d_input", shape=input_shape)
            mlmodel = ct.convert(model, inputs=[input_feature])
            mlmodel.save(f"mnist_cnn_model_coreML.mlpackage")
        except Exception as e:
            print(f"Error saving model {i}: {e}")
    
    # stop_event.set()  # 触发停止其他线程
    # raise ValueError

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save Core ML: {duration} seconds')

    # 测量推理时间
    X_test_sample = X_test[:10]  # 选择前10个样本进行推理
    start_time_inference = time.time()
    predictions = []
    for i in range(32):
        for sample in X_test_sample:
        # Core ML模型通常期望数据被添加一个额外的批次维度
            sample_reshaped = sample.reshape(1, 28, 28, 1)
            prediction = mlmodel.predict({"conv2d_input": sample_reshaped})
            predictions.append(prediction)
    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')



    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))


stop_event = threading.Event()
# model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")
model = tf.keras.models.load_model("./models_train/mnist_cnn_model.h5")


# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))

thread3.start()
thread1.start()
thread2.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# ----------------------------------------------------------------------------
file_name = f'./mnist_models/output_cnn/output-mnist-cnn-coreml.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 
# print("Extracted numbers:", numbers)

delta_time = duration * 2 / filtered_lines_count
# print(delta_time)

numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")


When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html


Saving model...
Subprocess started.


Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 1695.18 passes/s]
When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 1565.33 passes/s]


Time taken to save Core ML: 2.456733226776123 seconds
Time taken for inference on 10 samples: 0.1596 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./mnist_models/output_cnn/output-mnist-cnn-coreml.txt
276
['CPU Power: 5565 mW', 'GPU Power: 265 mW', 'CPU Power: 5101 mW', 'GPU Power: 187 mW', 'CPU Power: 5057 mW', 'GPU Power: 131 mW', 'CPU Power: 5694 mW', 'GPU Power: 134 mW', 'CPU Power: 6522 mW', 'GPU Power: 127 mW', 'CPU Power: 6224 mW', 'GPU Power: 64 mW', 'CPU Power: 5999 mW', 'GPU Power: 63 mW', 'CPU Power: 5002 mW', 'GPU Power: 0 mW', 'CPU Power: 4999 mW', 'GPU Power: 0 mW', 'CPU Power: 3895 mW', 'GPU Power: 0 mW', 'CPU Power: 4683 mW', 'GPU Power: 0 mW', 'CPU Power: 3302 mW', 'GPU Power: 0 mW', 'CPU Power: 2895 mW', 'GPU Power: 0 mW', 'CPU Power: 4882 mW', 'GPU Power: 0 mW', 'CPU Power: 6458 mW', 'GPU Power: 0 mW', 'CPU Power: 9643 mW', 'GPU Power: 0 mW', 'CPU Power: 6853 mW', 'GPU Power: 63 mW', 'CPU Power: 5340 mW', 'GPU Power: 373 mW', 'CPU Power:

In [8]:
import threading
import time
import psutil
import subprocess
import tempfile
import coremltools as ct
import tensorflow as tf
from tensorflow.keras.datasets import mnist

thread_output = {}
duration = 0
inference_duration = 0  # 添加推理时间变量

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    start_time = time.time()

    # 将TensorFlow Keras模型转换为Core ML模型
    for i in range(10):
        try:
            input_shape = ct.Shape(shape=(1, 28, 28, 1))
            input_feature = ct.TensorType(name="conv2d_input", shape=input_shape)
            mlmodel = ct.convert(model, inputs=[input_feature])
            mlmodel.save(f"mnist_cnn_model.mlpackage")
        except Exception as e:
            print(f"Error saving model {i}: {e}")

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save Core ML: {duration} seconds')

    # 测量推理时间
    X_test_sample = X_test[:10]  # 选择前10个样本进行推理
    start_time_inference = time.time()
    predictions = []
    for sample in X_test_sample:
        sample_reshaped = sample.reshape(1, 28, 28, 1)
        prediction = mlmodel.predict({"conv2d_input": sample_reshaped})
        predictions.append(prediction)
    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程


# 载入数据和模型
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

stop_event = threading.Event()
model = tf.keras.models.load_model("./mnist_models/mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))

thread3.start()
thread1.start()
thread2.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

file_name = f'./mnist_models/output_cnn/output-mnist-cnn-coreml.txt'
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')  # 将推理时间也写入文件
print(f"Content saved to {file_name}")


Saving model...
Subprocess started.


When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html
Running TensorFlow Graph Passes: 100%|██████████| 6/6 [00:00<00:00, 25.15 passes/s]
When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see 

Error saving model 0: Input (conv2d_input) provided is not found in given tensorflow graph. Placeholders in graph are: ['conv2d_2_input']


Running TensorFlow Graph Passes: 100%|██████████| 6/6 [00:00<00:00, 25.72 passes/s]
When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html


Error saving model 1: Input (conv2d_input) provided is not found in given tensorflow graph. Placeholders in graph are: ['conv2d_2_input']


...

Running TensorFlow Graph Passes: 100%|██████████| 6/6 [00:00<00:00, 25.63 passes/s]
When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html


Error saving model 8: Input (conv2d_input) provided is not found in given tensorflow graph. Placeholders in graph are: ['conv2d_2_input']


Running TensorFlow Graph Passes: 100%|██████████| 6/6 [00:00<00:00, 25.80 passes/s]
Exception in thread Thread-14:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/env_name/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/env_name/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/tm/w_n0tvhd2jlds4ml_jrhf3yw0000gn/T/ipykernel_9654/3087974962.py", line 60, in save_model
UnboundLocalError: local variable 'mlmodel' referenced before assignment


Error saving model 9: Input (conv2d_input) provided is not found in given tensorflow graph. Placeholders in graph are: ['conv2d_2_input']
Time taken to save Core ML: 6.9740540981292725 seconds


KeyboardInterrupt: 

CNN-CIFAR-10

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import cifar10  # Import CIFAR-10 dataset
from tensorflow.keras.utils import to_categorical

# 加载 CIFAR-10 数据集
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# 调整输入数据的形状和范围
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# 将标签转换为 one-hot 编码
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 创建 CNN 模型，修改 input_shape 以适应 CIFAR-10 的图像尺寸和通道数
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)),  # 注意修改 input_shape
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, batch_size=128, epochs=2, verbose=1, validation_data=(X_test, y_test))  # 可以考虑增加训练的周期

# 评估模型
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save('./models_train/cifar10_cnn_model.h5')

2024-07-25 20:59:56.612287: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-25 21:00:00.168088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/2
391/391 [==============================] - 12s 30ms/step - loss: 1.6810 - accuracy: 0.3878 - val_loss: 1.3402 - val_accuracy: 0.5331
Epoch 2/2
391/391 [==============================] - 11s 29ms/step - loss: 1.3573 - accuracy: 0.5161 - val_loss: 1.1581 - val_accuracy: 0.5921
Test loss: 1.1580811738967896
Test accuracy: 0.5921000242233276


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import cifar10  # Import CIFAR-10 dataset
from tensorflow.keras.utils import to_categorical
import coremltools as ct
import threading
import psutil
import subprocess
import time
import os
import tempfile
import re

cnn-cifar-10-H5

In [5]:
# 准备保存模型和监控资源使用的代码
thread_output = {}
duration = 0

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    
    # 测量保存模型的时间
    start_time = time.time()
    for i in range(0, 30):
        model.save("./models_train/cifar10_cnn_model.h5")
        
    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save TensorFlow SavedModel: {duration} seconds')
    stop_event.set()  # 触发停止其他线程

    # 测量推理时间
    X_test_sample = X_test[:10]  # 选择前10个样本进行推理
    start_time_inference = time.time()
    model.predict(X_test_sample)
    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

stop_event = threading.Event()
# model = tf.keras.models.load_model("./Cifar10_models/cifar10_cnn_model.h5")
model = tf.keras.models.load_model("./models_train/cifar10_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread3.start()
thread1.start()
thread2.start()
# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "GPU Power:" 或 "CPU Power:" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# 保存过滤后的内容和时长信息到文件
file_name = f'./Cifar10_models/output_cnn/output-cifar10-cnn-h5.txt'
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {file_name}")

# 计算和输出每个采样点的电量使用
filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 

delta_time = duration * 2 / filtered_lines_count
numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")

Saving model...
Subprocess started.


Exception in thread Thread-9:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/env_name/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/env_name/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/tm/w_n0tvhd2jlds4ml_jrhf3yw0000gn/T/ipykernel_56225/988918258.py", line 41, in save_model
NameError: name 'X_test' is not defined


Time taken to save TensorFlow SavedModel: 1.2089600563049316 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./Cifar10_models/output_cnn/output-cifar10-cnn-h5.txt
78
['CPU Power: 6313 mW', 'GPU Power: 0 mW', 'CPU Power: 5393 mW', 'GPU Power: 0 mW', 'CPU Power: 4541 mW', 'GPU Power: 0 mW', 'CPU Power: 5642 mW', 'GPU Power: 0 mW', 'CPU Power: 6508 mW', 'GPU Power: 0 mW', 'CPU Power: 8792 mW', 'GPU Power: 0 mW', 'CPU Power: 9147 mW', 'GPU Power: 0 mW', 'CPU Power: 8676 mW', 'GPU Power: 0 mW', 'CPU Power: 8347 mW', 'GPU Power: 0 mW', 'CPU Power: 5929 mW', 'GPU Power: 0 mW', 'CPU Power: 3887 mW', 'GPU Power: 0 mW', 'CPU Power: 6175 mW', 'GPU Power: 0 mW', 'CPU Power: 6317 mW', 'GPU Power: 0 mW', 'CPU Power: 6211 mW', 'GPU Power: 0 mW', 'CPU Power: 8082 mW', 'GPU Power: 0 mW', 'CPU Power: 8057 mW', 'GPU Power: 0 mW', 'CPU Power: 8765 mW', 'GPU Power: 0 mW', 'CPU Power: 8481 mW', 'GPU Power: 0 mW', 'CPU Power: 7880 mW', 'GPU Power: 0 mW', 'CPU Power: 6133 mW', 'GPU

CNN-cifar-10-ONNX

In [6]:
thread_output = {}
duration = 0
inference_duration = 0

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None
        time.sleep(0.1)

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    # --------------------------------------------------------------------------------
    start_time = time.time()
    
    # 使用 tf2onnx 保存模型为ONNX格式
    spec = (tf.TensorSpec((None, 32, 32, 3), tf.float32, name="input"),)
    output_path = './Cifar10_models/cifar10_cnn_model.onnx'
    
    for i in range(20):
        model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=output_path)
    
    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save ONNX model: {duration} seconds')
    # --------------------------------------------------------------------------------
    
    # 测量推理时间
    X_test_sample = X_test[:10]  # 选择前10个样本进行推理
    start_time_inference = time.time()
    model.predict(X_test_sample)
    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
                time.sleep(0.1)
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=5)  # 尝试在5秒内等待进程结束
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

stop_event = threading.Event()

model = tf.keras.models.load_model("./models_train/cifar10_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread3.start()
thread1.start()
thread2.start()
# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "GPU Power:" 或 "CPU Power:" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

file_name = './Cifar10_models/output_cnn/output-cifar10-cnn-onnx.txt'
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 

delta_time = duration * 2 / filtered_lines_count
numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")

Saving model...
Subprocess started.


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/env_name/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/env_name/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/tm/w_n0tvhd2jlds4ml_jrhf3yw0000gn/T/ipykernel_56225/1374259615.py", line 45, in save_model
NameError: name 'X_test' is not defined


Time taken to save ONNX model: 5.29556679725647 seconds


KeyboardInterrupt: 

: 

CNN-cifar-core ml

In [15]:
thread_output = {}
duration = 0
inference_duration = 0

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None
        # time.sleep(0.1)

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    # --------------------------------------------------------------------------------
    start_time = time.time()

    # 将TensorFlow Keras模型转换为Core ML模型
    for i in range(1):
        try:
            input_shape = ct.Shape(shape=(1, 32, 32, 3))
            input_feature = ct.TensorType(name="conv2d_input", shape=input_shape)
            mlmodel = ct.convert(model, inputs=[input_feature])
            mlmodel.save(f"cifar10_cnn_model.mlpackage")
        except Exception as e:
            print(f"Error saving model {i}: {e}")
    # --------------------------------------------------------------------------------

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save core-ml model: {duration} seconds')
    
    # 测量推理时间
    X_test_sample = X_test[:10]  # 选择前10个样本进行推理
    start_time_inference = time.time()
    model.predict(X_test_sample)
    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
                # time.sleep(0.1)
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    # process.wait(timeout=5)  # 尝试在5秒内等待进程结束
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 载入数据和模型

stop_event = threading.Event()
model = tf.keras.models.load_model("./models_train/cifar10_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread3.start()
thread1.start()
thread2.start()
# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')] # or
                  # line.startswith('Combined Power (CPU + GPU + ANE):')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# ----------------------------------------------------------------------------
file_name = f'./Cifar10_models//output_cnn/output-cifar10-cnn-coreML.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 
# print("Extracted numbers:", numbers)

delta_time = duration * 2 / filtered_lines_count
# print(delta_time)

numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption} mV")

Saving model...
Subprocess started.


2024-07-25 21:10:53.073805: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-07-25 21:10:53.456136: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html
2024-07-25 21:10:53.961465: I tensorflow/core/grappler/devices.cc:75] Number of eligib

Time taken to save core-ml model: 9.0000901222229 seconds
1/1 [==============================] - 1s 996ms/step
Time taken for inference on 10 samples: 1.8422 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./Cifar10_models//output_cnn/output-cifar10-cnn-coreML.txt
588
['CPU Power: 5868 mW', 'GPU Power: 30 mW', 'CPU Power: 4508 mW', 'GPU Power: 0 mW', 'CPU Power: 5378 mW', 'GPU Power: 0 mW', 'CPU Power: 5011 mW', 'GPU Power: 66 mW', 'CPU Power: 5595 mW', 'GPU Power: 34 mW', 'CPU Power: 5311 mW', 'GPU Power: 31 mW', 'CPU Power: 4905 mW', 'GPU Power: 95 mW', 'CPU Power: 5026 mW', 'GPU Power: 98 mW', 'CPU Power: 5327 mW', 'GPU Power: 34 mW', 'CPU Power: 4623 mW', 'GPU Power: 31 mW', 'CPU Power: 5580 mW', 'GPU Power: 0 mW', 'CPU Power: 5004 mW', 'GPU Power: 30 mW', 'CPU Power: 4293 mW', 'GPU Power: 29 mW', 'CPU Power: 5969 mW', 'GPU Power: 36 mW', 'CPU Power: 6012 mW', 'GPU Power: 32 mW', 'CPU Power: 6230 mW', 'GPU Power: 0 mW', 'CPU Power: 4441 mW', 'GPU Power: 

Subprocess finished.


### cifar10 cnn pytorch

In [17]:
thread_output = {}
duration = 0
inference_duration = 0  # 添加用于存储推理时间的变量

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    start_time = time.time()

    # 保存整个模型为 PyTorch 格式
    for i in range(5):
        torch.save(model, 'pytorch_model.pth')

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save PyTorch model: {duration:.2f} seconds')

    # 测量推理时间
    X_test_sample = X_test[:10]  # 选择前10个样本进行推理
    start_time_inference = time.time()
    model.predict(X_test_sample)
    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))


stop_event = threading.Event()
model = tf.keras.models.load_model("./models_train/cifar10_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))


thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
lines = content.split('\n')
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')]

filtered_content = '\n'.join(filtered_lines)

# ----------------------------------------------------------------------------
file_name = f'./Cifar10_models/output_cnn/output-cifar10-cnn-pth.txt'
# ----------------------------------------------------------------------------
with open(file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {file_name}")

filtered_lines_count = len(filtered_lines)
print(filtered_lines_count)
print(filtered_lines)

# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")

Saving model...
Subprocess started.
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dropout
.........vars
......flatten
.........vars
......max_pooling2d
.........vars
......max_pooling2d_1
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2024-07-25 21:16:18        